# Caravan-ESP: preprocesar caudal
***

***Autor:** Jesús Casado Rodríguez*<br>
***Fecha:** 05-08-2023*<br>

**Introducción:**<br>

In [2]:
import os
os.environ['USE_PYGEOS'] = '0'
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import xarray as xr
import geopandas as gpd
from tqdm.notebook import tqdm

In [78]:
path_caravan = 'C:/Users/jcr693/Documents/Caravan-ESP/ESP/'

In [79]:
# cargar tabla de atributos de Caravan
attributes = pd.read_csv(f'{path_caravan}output/attributes/esp/attributes_caravan_esp.csv', index_col=0)
attributes.index = attributes.index.astype(str)

In [80]:
# cargar tabla de las estaciones seleccionadas para Caravan
stations = pd.read_csv(f'{path_caravan}caravan_estaciones.csv', index_col='indroea')
stations.index = stations.index.astype(str)
# recortar estaciones a la tabla de atributos
stations = stations.loc[attributes.index, :]

In [81]:
# cargar polígonos de las cuencas de Caravan
catchments = gpd.read_file('C:/Users/jcr693/Documents/GitHub/TFM/data/GIS/subcuencas_epsg4326.shp')
catchments.set_index('gauge_id', drop=False, inplace=True)
catchments.index = catchments.index.astype(str)
# recortar cuencas según la tabla de atributos
catchments = catchments.loc[attributes.index, :]

# generar nuevos campos
catchments['gauge_name'] = stations.lugar
catchments['gauge_lat'] = stations.latwgs84 / 1e4
catchments['gauge_lon'] = stations.longwgs84 / 1e4
catchments['country'] = 'Spain'
catchments['area_skm'] = catchments.Shape_Area * 1e-6
catchments.area_skm = catchments.area_skm.astype(int)

# eliminar campos
catchments.drop(['gauge_id', 'OID_', 'Name', 'Shape_Leng', 'Shape_Area'], axis=1, inplace=True)

# reordenar
catchments = catchments[['gauge_name', 'gauge_lat', 'gauge_lon', 'country', 'area_skm', 'geometry']]
catchments.head()

,gauge_name,gauge_lat,gauge_lon,country,area_skm,geometry
gauge_id,,,,,,
1105,EREÑOZU,43.1437,-1.5630,Spain,215,"POLYGON ((-1.90497 43.25597, -1.90499 43.25527..."
1163,LEMONA (IBAIZABAL),43.1242,-2.4518,Spain,252,"POLYGON ((-2.55984 43.06733, -2.56176 43.06734..."
1186,GURIEZO,43.2116,-3.1947,Spain,117,"POLYGON ((-3.27725 43.19324, -3.27821 43.19324..."
1196,COTERILLO,43.1948,-3.2610,Spain,493,"POLYGON ((-3.57330 43.34361, -3.57329 43.34291..."
1268,PUENTELLES,43.1902,-4.3626,Spain,650,"POLYGON ((-4.60254 43.31762, -4.60252 43.31691..."


In [82]:
# exportar
catchments.to_file(f'{path_caravan}output/shapefiles/esp/catchments_caravan_esp.shp', driver='ESRI Shapefile')